# America's 500 Largest Cities through the Numbers
### An exploration of urban health compiled by Kennon Stewart
We live in strange times. The COVID-19 pandemic exposed the holes in many states' health systems and, for the first time, states are reckoning with their inadequate public health measures. This is an exploration of the pre-COVID health issues facing individual census tracts in 2016-2017, brought together to paint a larger picture of their state.

If you want to hear the playlist that drove me during this project, check <a href= 'https://open.spotify.com/playlist/1Cqwlw4s5SZ5SxssoW1f2D?si=b11jPdcbQKCIsQCrYEPsiA'>here</a>.

In [1]:
%autosave 15
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

Autosaving every 15 seconds


The dataset I'm using is the 500 Cities database provided by the CDC in 2019. Unfortunately the set only has data from years 2016 and 2017 so, if anyone knows of more comprehensive data, let me know! As for this dataset, I want to save as much stress on my laptop as possible so I'll only read the relevant columns.

In [2]:
cols = ['Year','StateAbbr','High_Confidence_Limit','UniqueID','Short_Question_Text']

# Variables:
# StateAbbr = State Abbreviation
# High_Confidence_Limit = the highest confidence limit estimate for a particular census tract for a particular ailment
# Unique_ID = the unique ID for each census tract in the country which will be matched with census data from the year 2017
# Short_Question_Text = 

cities = pd.read_csv('/Users/student/Downloads/500 Cities/500_Cities__Local_Data_for_Better_Health__2019_release.csv', usecols=cols)

In [4]:
cols2 = ['GEO_ID','DP05_0038E','DP05_0038M','DP05_0038PE','DP05_0038PM']

# Variables:
# GEO_ID = ID of the Census tract 
# DP05_0038E = estimated real count of Black Americans
# DP05_0038M = margin of error of real count of Black Americans
# DP05_0038PE = estimated percentage of Black Americans
# DP05_0038PM = margin of error of estimated percentage of Black Americans

racial = pd.read_csv('/Users/student/Downloads/500 Cities/ACSDP5Y2017.DP05_2020-06-27T105923/ACSDP5Y2017.DP05_data_with_overlays_2020-06-27T105738.csv',usecols=cols2,low_memory=False)

It also looks like we've got some useful ailments to examine. In the theme of common illnesses, I'm going to look at Current Asthma, High Cholesterol, High Blood Pressure, and Physical Health. These are, conveniently, features for which all 29006 census tracts have been surveyed.

In [5]:
cities['ID'] = cities['UniqueID'].map(lambda x: x.replace('-',''))
cities['ID'] = cities['UniqueID'].map(lambda x: x[8:])
cities = cities.drop('UniqueID',axis=1)

In [6]:
cities.head(10)

,Year,StateAbbr,High_Confidence_Limit,Short_Question_Text,ID
0,2016,IL,39.9,Teeth Loss,17031612000
1,2016,IL,46.3,Dental Visit,17031660700
2,2017,IL,10.5,Current Asthma,17115001100
3,2016,IL,27.4,Teeth Loss,17197881603
4,2017,IL,26.3,High Cholesterol,17031804405
5,2016,IL,28.3,Teeth Loss,17197882801
6,2016,IL,37.4,Sleep <7 hours,17031840400
7,2017,IL,10.3,Diabetes,17031151002
8,2017,IL,41.1,Obesity,17031381900
9,2016,IL,14.1,Teeth Loss,17031130300


In [7]:
racial = racial.drop([0])
racial['ID'] = racial['GEO_ID'].map(lambda x: x.replace('US',''))
mess = ['*','**','-']
for i in mess:
    racial['DP05_0038PM'] = racial['DP05_0038PM'].map(lambda x: x.replace(i,'NaN'))
    racial['DP05_0038PE'] = racial['DP05_0038PE'].map(lambda y: y.replace(i,'NaN'))
racial['ID'] = racial['GEO_ID'].map(lambda x: x[9:])
racial = racial.drop('GEO_ID',axis=1)

In [8]:
racial.head()

,DP05_0038E,DP05_0038M,DP05_0038PE,DP05_0038PM,ID
1,163,161,6.2,5.9,01007010001
2,303,199,4.9,3.1,01007010002
3,890,412,17.7,7.3,01007010003
4,3623,438,41.6,4.2,01007010004
5,980,151,87.5,6.6,01087231603


We were successfully able to merge the racial and health data of most census tracts within the United States, consisting of the 500 largest cities within the country. Because of the outer merge, we can also see that some of the census data we loaded doesn't have any corresponding CDC data. Since the analysis is only of the 500 largest cities, we'll exclude these from analysis. In addition, some of the survey tracts within the 500 Cities survey were too small to be surveyed effectively, so researchers didn't report those counties.

In [9]:
combined = pd.merge(racial, cities, on='ID', how='inner')
combined.dropna(inplace=True)

In [10]:
combined.shape

(746577, 9)

In [11]:
michigan = combined[combined['StateAbbr']=='MI']

I'm a fan of statistics and so I wanted to run an experiment on my home state of Michigan. Now that I have the percentage of Black residents in each census tract as well as the prevalence of a certain health issue in that tract, I can run a one-way ANOVA. This is a standard test to determine whether an independent variable impacts some numeric dependent variable, in this instance we'll do Coronary Heart Disease.

In [12]:
chdMI = michigan[michigan['Short_Question_Text']=='Coronary Heart Disease']

In [13]:
chdMI.head(15)

,DP05_0038E,DP05_0038M,DP05_0038PE,DP05_0038PM,ID,Year,StateAbbr,High_Confidence_Limit,Short_Question_Text
378930,236,207,3.7,3.2,26099230602,2017,MI,8.0,Coronary Heart Disease
378977,197,114,2.8,1.6,26099230601,2017,MI,7.0,Coronary Heart Disease
378996,942,275,26.8,6.0,26099268400,2017,MI,10.5,Coronary Heart Disease
379102,353,116,4.7,1.6,26161400300,2017,MI,1.8,Coronary Heart Disease
379123,88,82,2.0,1.8,26161400600,2017,MI,3.4,Coronary Heart Disease
379155,439,170,21.2,7.3,26161404200,2017,MI,4.3,Coronary Heart Disease
379184,4180,472,96.0,2.7,26163541500,2017,MI,10.6,Coronary Heart Disease
379217,2092,290,93.9,3.1,26163541700,2017,MI,10.5,Coronary Heart Disease
379236,3017,427,95.1,5.5,26163543200,2017,MI,7.9,Coronary Heart Disease
379275,1479,371,96.0,2.6,26163545300,2017,MI,10.6,Coronary Heart Disease


We know that the dependent variable is approximately normally distributed because of the histogram of the High Confidence Limit. This satisfies the requirement of normality for the two-way Analysis of Variance (ANOVA) test.

In [15]:
chdMI[['DP05_0038E','DP05_0038M']] = chdMI[['DP05_0038E','DP05_0038M']].astype(int)
chdMI[['DP05_0038PE','DP05_0038PM']] = chdMI[['DP05_0038PE','DP05_0038PM']].astype('float64')
chdMI['DP05_0038PE'].describe()

count    724.000000
mean      44.781906
std       38.695059
min        0.000000
25%        7.900000
50%       27.800000
75%       89.875000
max      100.000000
Name: DP05_0038PE, dtype: float64

In [16]:
chdMI.loc[chdMI['DP05_0038PE'] <= 100, 'DL']= 4
chdMI.loc[chdMI['DP05_0038PE'] <= 89.875, 'DL']= 3
chdMI.loc[chdMI['DP05_0038PE'] <= 27.8, 'DL']= 2
chdMI.loc[chdMI['DP05_0038PE'] <= 7.9, 'DL']= 1

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
chdMI['DL'] = chdMI['DL'].astype('int')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
chdMI.head()

,DP05_0038E,DP05_0038M,DP05_0038PE,DP05_0038PM,ID,Year,StateAbbr,High_Confidence_Limit,Short_Question_Text,DL
378930,236,207,3.7,3.2,26099230602,2017,MI,8.0,Coronary Heart Disease,1
378977,197,114,2.8,1.6,26099230601,2017,MI,7.0,Coronary Heart Disease,1
378996,942,275,26.8,6.0,26099268400,2017,MI,10.5,Coronary Heart Disease,2
379102,353,116,4.7,1.6,26161400300,2017,MI,1.8,Coronary Heart Disease,1
379123,88,82,2.0,1.8,26161400600,2017,MI,3.4,Coronary Heart Disease,1


### A Note on Categorical Variables
Created categorical variable for every census tract based on their percentage Black population in relation to the rest of their state. Categories are 1, 2, 3, and 4 with 1 being 1 being a tract in the lower 25% of Michigan tracts for % of Black residents and 4 being the highest 25% of Michigan tracts for Black residents. This allows us to rank categorical variables and continue with statistical tests that require them.